<a href="https://colab.research.google.com/github/21WH1A0422Ramameghana/ML-Project-Team5-/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):

  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

In [ ]:
# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):

  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
  async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  img = js_to_image(data)
  # grayscale img
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  print(gray.shape)
  # get face bounding box coordinates using Haar Cascade
  faces = face_cascade.detectMultiScale(gray)
  # draw face bounding box on image
#   for (x,y,w,h) in faces:
#       img = cv2.rectangle(img,(x-50,y-50),(x+w+70,y+h+50),(255,0,0),2)
#   # save image
  cv2.imwrite(filename, img)


In [ ]:
try:
  filename = take_photo('snapshot.jpg')
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

<IPython.core.display.Javascript object>

'NoneType' object has no attribute 'split'


In [ ]:
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 16.1 MB/s eta 0:00:00


In [ ]:
import easyocr
import re

# Load your image
image_path = 'snapshot.jpg'

# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'])  # Use English language for text recognition

# Perform text extraction
results = reader.readtext(image_path)

# Filter extracted text to keep only 6-digit numbers
six_digit_numbers = []
for result in results:
    text = result[1]
    # Use a regular expression to match 6-digit numbers
    match = re.match(r'\d{6}', text)
    if match:
      if len(str(text)) == 6:
        six_digit_numbers.append(text)

# Print the extracted 6-digit numbers
for number in six_digit_numbers:
  #
    print("Extracted 6-digit number:", number)



Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteExtracted 6-digit number: 505467


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the CSV file containing pin codes, district names, and state names
data = pd.read_csv('/content/drive/MyDrive/pincode-dataset.csv')
# Assuming 'data' is your DataFrame
# Impute missing values in numerical columns with the mean
numerical_columns = data.select_dtypes(include=['number']).columns
data[numerical_columns] = data[numerical_columns].fillna(data[numerical_columns].mean())

# Impute missing values in categorical columns with the mode
categorical_columns = data.select_dtypes(exclude=['number']).columns
data[categorical_columns] = data[categorical_columns].fillna(data[categorical_columns].mode().iloc[0])


# Prepare the dataset
X = data['Pincode'].values.reshape(-1, 1)  # Features (6-digit numbers)
y_district = data['District'].values  # District names (target)
y_state = data['StateName'].values  # State names (target)

# Split the dataset into training and testing sets
X_train, X_test, y_district_train, y_district_test, y_state_train, y_state_test = train_test_split(
    X, y_district, y_state, test_size=0.2, random_state=42
)

# Train a Random Forest classifier for district names
district_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
district_classifier.fit(X_train, y_district_train)

# Train a Random Forest classifier for state names
state_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
state_classifier.fit(X_train, y_state_train)

# Predict district and state names for test data
district_predictions = district_classifier.predict(X_test)
state_predictions = state_classifier.predict(X_test)

# Calculate accuracy for district and state predictions
district_accuracy = accuracy_score(y_district_test, district_predictions)
state_accuracy = accuracy_score(y_state_test, state_predictions)

print("District Classifier Accuracy:", district_accuracy)
print("State Classifier Accuracy:", state_accuracy)

# Reshape the input array to be a 2D array with a single feature
six_digit_numbers = [[int(num)] for num in six_digit_numbers]

district_predictions_example = district_classifier.predict( six_digit_numbers)
state_predictions_example = state_classifier.predict( six_digit_numbers)

for i in range(len(six_digit_numbers)):
    print(f"Pincode: {six_digit_numbers[i][0]}, Predicted District: {district_predictions_example[i]}, Predicted State: {state_predictions_example[i]}")


District Classifier Accuracy: 0.8488704232666839
State Classifier Accuracy: 0.9963645806284082
Pincode: 505467, Predicted District: SIDDIPET, Predicted State: Telangana
